In [1]:
ACTION_DESCRIPTIONS = [
    {'index': 0, 'object': 'right_hip', 'min': -1, 'max': 1, 'object_type': 'motor', 'action_type': 'continuous', 'left_actor': True, 'right_actor': False},
    {'index': 1, 'object': 'right_knee', 'min': -1, 'max': 1, 'object_type': 'motor', 'action_type': 'continuous', 'left_actor': True, 'right_actor': False},
    {'index': 2, 'object': 'right_ankle', 'min': -1, 'max': 1, 'object_type': 'motor', 'action_type': 'continuous', 'left_actor': True, 'right_actor': False},
    {'index': 3, 'object': 'left_hip', 'min': -1, 'max': 1, 'object_type': 'motor', 'action_type': 'continuous', 'left_actor': False, 'right_actor': True},
    {'index': 4, 'object': 'left_knee', 'min': -1, 'max': 1, 'object_type': 'motor', 'action_type': 'continuous', 'left_actor': False, 'right_actor': True},
    {'index': 5, 'object': 'left_ankle', 'min': -1, 'max': 1, 'object_type': 'motor', 'action_type': 'continuous', 'left_actor': False, 'right_actor': True}]
OBSERVATION_DESCRIPTIONS = [
    {'index': 0, 'object': 'torso', 'object_type': 'orientation', 'axis': 'xx', 'left_actor': True, 'right_actor': True},
    {'index': 1, 'object': 'torso', 'object_type': 'orientation', 'axis': 'xz', 'left_actor': True, 'right_actor': True},
    {'index': 2, 'object': 'right_thigh', 'object_type': 'orientation', 'axis': 'xx', 'left_actor': True, 'right_actor': False},
    {'index': 3, 'object': 'right_thigh', 'object_type': 'orientation', 'axis': 'xz', 'left_actor': True, 'right_actor': False},
    {'index': 4, 'object': 'right_leg', 'object_type': 'orientation', 'axis': 'xx', 'left_actor': True, 'right_actor': False},
    {'index': 5, 'object': 'right_leg', 'object_type': 'orientation', 'axis': 'xz', 'left_actor': True, 'right_actor': False},
    {'index': 6, 'object': 'right_foot', 'object_type': 'orientation', 'axis': 'xx', 'left_actor': True, 'right_actor': False},
    {'index': 7, 'object': 'right_foot', 'object_type': 'orientation', 'axis': 'xz', 'left_actor': True, 'right_actor': False},
    {'index': 8, 'object': 'left_thigh', 'object_type': 'orientation', 'axis': 'xx', 'left_actor': False, 'right_actor': True},
    {'index': 9, 'object': 'left_thigh', 'object_type': 'orientation', 'axis': 'xz', 'left_actor': False, 'right_actor': True},
    {'index': 10, 'object': 'left_leg', 'object_type': 'orientation', 'axis': 'xx', 'left_actor': False, 'right_actor': True},
    {'index': 11, 'object': 'left_leg', 'object_type': 'orientation', 'axis': 'xz', 'left_actor': False, 'right_actor': True},
    {'index': 12, 'object': 'left_foot', 'object_type': 'orientation', 'axis': 'xx', 'left_actor': False, 'right_actor': True},
    {'index': 13, 'object': 'left_foot', 'object_type': 'orientation', 'axis': 'xz', 'left_actor': False, 'right_actor': True},
    {'index': 14, 'object': 'torso', 'object_type': 'height', 'left_actor': True, 'right_actor': True},
    {'index': 15, 'object': 'rootz', 'object_type': 'velocity', 'left_actor': True, 'right_actor': True},
    {'index': 16, 'object': 'rootx', 'object_type': 'velocity', 'left_actor': True, 'right_actor': True},
    {'index': 17, 'object': 'rooty', 'object_type': 'velocity', 'left_actor': True, 'right_actor': True},
    {'index': 18, 'object': 'right_hip', 'object_type': 'velocity', 'left_actor': True, 'right_actor': False},
    {'index': 19, 'object': 'right_knee', 'object_type': 'velocity', 'left_actor': True, 'right_actor': False},
    {'index': 20, 'object': 'right_ankle', 'object_type': 'velocity', 'left_actor': True, 'right_actor': False},
    {'index': 21, 'object': 'left_hip', 'object_type': 'velocity', 'left_actor': False, 'right_actor': True},
    {'index': 22, 'object': 'left_knee', 'object_type': 'velocity', 'left_actor': False, 'right_actor': True},
    {'index': 23, 'object': 'left_ankle', 'object_type': 'velocity', 'left_actor': False, 'right_actor': True},
]

In [2]:
import os
os.environ['MUJOCO_GL'] = 'egl'
os.environ['LAZY_LEGACY_OP'] = '0'
import warnings
warnings.filterwarnings('ignore')
import torch

import hydra
from termcolor import colored

from common.parser import parse_cfg
from common.seed import set_seed
from common.buffer import Buffer
from envs import make_env
from dialectic import DialecticMPC, DialecticImitation
from trainer.offline_trainer import OfflineTrainer
from trainer.online_trainer import OnlineTrainer
from common.logger import Logger

torch.backends.cudnn.benchmark = True

torch.cuda.is_available()

True

In [3]:
import yaml

def read_yaml_to_dict(file_path):
    try:
        with open(file_path, 'r') as file:
            yaml_data = yaml.safe_load(file)
        return yaml_data
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {file_path}")
        return None
    except yaml.YAMLError as e:
        print(f"YAML 파싱 오류: {e}")
        return None

In [4]:
# 사용 예시
file_path = 'dialectic_config.yaml'
result = read_yaml_to_dict(file_path)

if result:
    print("YAML 내용:")
    print(result)

YAML 내용:
{'defaults': [{'override hydra/launcher': 'submitit_local'}], 'task': 'walker-walk', 'obs': 'state', 'checkpoint': '???', 'eval_episodes': 10, 'eval_freq': 5000, 'device': 'cuda:0', 'steps': 10000000, 'train_per_steps': 1000, 'batch_size': 256, 'reward_coef': 0.1, 'value_coef': 0.1, 'consistency_coef': 20, 'rho': 0.5, 'lr': '3e-5', 'enc_lr_scale': 0.3, 'grad_clip_norm': 20, 'tau': 0.01, 'discount_denom': 5, 'discount_min': 0.95, 'discount_max': 0.995, 'discount_gamma': 0.99, 'buffer_size': 1000000, 'exp_name': 'default', 'data_dir': '???', 'mpc': True, 'iterations': 6, 'num_samples': 512, 'num_elites': 64, 'num_pi_trajs': 24, 'horizon': 3, 'min_std': 0.05, 'max_std': 2, 'temperature': 0.5, 'log_std_min': -10, 'log_std_max': 2, 'entropy_coef': '1e-4', 'num_bins': 101, 'vmin': -10, 'vmax': 10, 'model_size': '???', 'num_enc_layers': 2, 'enc_dim': 256, 'num_channels': 32, 'mlp_dim': 512, 'mlp_layers': 2, 'latent_dim': 512, 'task_dim': 96, 'num_q': 5, 'dropout': 0.01, 'simnorm_dim'

In [5]:
from omegaconf import OmegaConf
cfg = OmegaConf.load('dialectic_config.yaml')

In [6]:
from pathlib import Path
work_dir = Path('/home/pipaek/project/tdmpc2') / 'logs' / cfg.task / str(cfg.seed) / cfg.exp_name

cfg.work_dir = work_dir
work_dir

PosixPath('/home/pipaek/project/tdmpc2/logs/walker-walk/1/default')

In [7]:
from common import MODEL_SIZE, TASK_SET

def myparse_cfg(cfg: OmegaConf) -> OmegaConf:
	"""
	Parses a Hydra config. Mostly for convenience.
	"""

	# Logic
	for k in cfg.keys():
		try:
			v = cfg[k]
			if v == None:
				v = True
		except:
			pass

	# Algebraic expressions
	for k in cfg.keys():
		try:
			v = cfg[k]
			if isinstance(v, str):
				match = re.match(r"(\d+)([+\-*/])(\d+)", v)
				if match:
					cfg[k] = eval(match.group(1) + match.group(2) + match.group(3))
					if isinstance(cfg[k], float) and cfg[k].is_integer():
						cfg[k] = int(cfg[k])
		except:
			pass

	# Convenience
	cfg.task_title = cfg.task.replace("-", " ").title()
	cfg.bin_size = (cfg.vmax - cfg.vmin) / (cfg.num_bins-1) # Bin size for discrete regression

	# Model size
	if cfg.get('model_size', None) is not None:
		assert cfg.model_size in MODEL_SIZE.keys(), \
			f'Invalid model size {cfg.model_size}. Must be one of {list(MODEL_SIZE.keys())}'
		for k, v in MODEL_SIZE[cfg.model_size].items():
			cfg[k] = v
		if cfg.task == 'mt30' and cfg.model_size == 19:
			cfg.latent_dim = 512 # This checkpoint is slightly smaller

	# Multi-task
	cfg.multitask = cfg.task in TASK_SET.keys()
	if cfg.multitask:
		cfg.task_title = cfg.task.upper()
		# Account for slight inconsistency in task_dim for the mt30 experiments
		cfg.task_dim = 96 if cfg.task == 'mt80' or cfg.model_size in {1, 317} else 64
	else:
		cfg.task_dim = 0
	cfg.tasks = TASK_SET.get(cfg.task, [cfg.task])

	return cfg

In [8]:
cfg = myparse_cfg(cfg)

In [9]:
set_seed(cfg.seed)

In [10]:
from trainer.online_trainer import OnlineTrainer, OnlineDialecticTrainer, OnlineDialecticImitationTrainer, OnlineSingleImitationTrainer, OnlineDialecticImitationTrainer
from reinforce import ReinforceAgent, DialecticReinforceAgent

trainer = OnlineDialecticImitationTrainer(
		cfg=cfg,
		env=make_env(cfg),
		agent=DialecticReinforceAgent(cfg),
		logger=Logger(cfg),
	)

------------------------------------
  Task:           Walker Walk
  Steps:          10,000,000
  Observations:   [24]
  Actions:        6
  Experiment:     default
------------------------------------
Wandb disabled.
Architecture: DualModel(
  (_brain_l): Sequential(
    (0): NormedLinear(in_features=18, out_features=512, bias=True, act=Mish)
    (1): NormedLinear(in_features=512, out_features=512, bias=True, act=Mish)
    (2): Linear(in_features=512, out_features=6, bias=True)
  )
  (_brain_r): Sequential(
    (0): NormedLinear(in_features=18, out_features=512, bias=True, act=Mish)
    (1): NormedLinear(in_features=512, out_features=512, bias=True, act=Mish)
    (2): Linear(in_features=512, out_features=6, bias=True)
  )
)
Learnable parameters: 555,020


In [13]:
self = trainer

In [14]:
obs = self.env.reset()

In [15]:
self._tds_l = []
self._tds_r = []

In [16]:
obs_total = obs
self.brain_switch = True
if self.brain_switch:  # left brain
    obs = obs[self.agent.obs_filter_l].to(self.agent.device, non_blocking=True).unsqueeze(0)
    brain = self.agent.model._brain_l
    last_opposite_action = self.agent.last_action_r.detach()
else:  # right brain
    obs = obs[self.agent.obs_filter_r].to(self.agent.device, non_blocking=True).unsqueeze(0)
    brain = self.agent.model._brain_r
    last_opposite_action = self.agent.last_action_l.detach()
if self.agent.obs_class == 'all':
    obs = obs_total.to(self.agent.device, non_blocking=True).unsqueeze(0)  ## 전체 스페이스로 테스트해보기
state = torch.concat([obs, last_opposite_action], dim=1)

In [20]:
state.shape

torch.Size([1, 18])

In [21]:
brain_l = self.agent.model._brain_l
brain_r = self.agent.model._brain_r

In [22]:
a = brain_l(state)
a

tensor([[-0.2138, -0.0394,  0.0433,  0.0043,  0.0628,  0.1641]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [26]:
action, mu, log_sigma = self.agent._calculate_action(a, self.agent.action_dim_l, eval_mode=False)

In [27]:
action

tensor([[-0.8505, -0.7242, -0.7131]], device='cuda:0', grad_fn=<AddBackward0>)

In [29]:
mu

tensor([[-0.2138, -0.0394,  0.0433]], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [31]:
log_sigma

tensor([[0.0043, 0.0628, 0.1641]], device='cuda:0', grad_fn=<SliceBackward0>)

In [32]:
done = False
obs = self.env.reset()
self._tds_l = []
self._tds_r = []
while not done:
    action_l, dist_l, value_l, state_l, pred, _ = self.agent.act(obs, t0=len(self._tds_l)==1)
    action_r, dist_r, value_r, state_r, pred, _ = self.agent.act(obs, t0=len(self._tds_r)==1)
    action = torch.concat([action_l, action_r], dim=1).detach()
    action_np = action[0].detach().cpu()
    obs, reward, done, info = self.env.step(action_np)
    done = torch.ones_like(reward) if done else torch.zeros_like(reward)
    self._tds_l.append(self.to_td(state_l, action_l, reward, dist_l[0], dist_l[1], value_l, done))
    self._tds_r.append(self.to_td(state_r, action_r, reward, dist_r[0], dist_r[1], value_r, done))

    self._step += 1
    
print('total steps:', self._step)
    

total steps: 500


In [ ]:
actions, rewards, mus, log_sigmas = self.agent.get_policy_train_data(self._tds_l)

In [ ]:
actions.shape

torch.Size([500, 1, 6])

In [ ]:
rewards.shape

torch.Size([500])

In [ ]:
mus.shape

torch.Size([500, 1, 3])

In [ ]:
log_sigmas.shape

torch.Size([500, 1, 3])

In [ ]:
actions = torch.cat([td['action'] for td in self._tds_l]).to(self.agent.device)
actions.shape

torch.Size([500, 1, 6])

In [ ]:
rewards = torch.cat([td['reward'] for td in self._tds])
rewards.shape

RuntimeError: torch.cat(): expected a non-empty list of Tensors

In [ ]:
mus = torch.cat([td['mu'] for td in self._tds])
mus.shape

torch.Size([500, 1, 6])

In [ ]:
log_sigmas = torch.cat([td['log_sigma'] for td in self._tds])
log_sigmas

tensor([[[-0.4407,  0.4089,  0.1255,  0.0720,  0.0891, -0.3190]],

        [[ 0.2812, -0.1259,  0.2329,  0.3431, -0.3246, -0.0381]],

        [[ 0.0602,  0.1832,  0.2661,  0.0746, -0.0328,  0.3918]],

        ...,

        [[-0.1648, -0.1007,  0.1023, -0.5227, -0.0170, -0.0587]],

        [[ 0.1351,  0.3486,  0.4247,  0.0562, -0.2721,  0.0755]],

        [[ 0.2081, -0.4042,  0.1990,  0.3789, -0.1175,  0.0147]]],
       device='cuda:0', grad_fn=<CatBackward0>)

In [ ]:
log_sigmas[0, 0, 0] = -0.4407  # -0.4407
log_sigmas


tensor([[[-0.4407,  0.4089,  0.1255,  0.0720,  0.0891, -0.3190]],

        [[ 0.2812, -0.1259,  0.2329,  0.3431, -0.3246, -0.0381]],

        [[ 0.0602,  0.1832,  0.2661,  0.0746, -0.0328,  0.3918]],

        ...,

        [[-0.1648, -0.1007,  0.1023, -0.5227, -0.0170, -0.0587]],

        [[ 0.1351,  0.3486,  0.4247,  0.0562, -0.2721,  0.0755]],

        [[ 0.2081, -0.4042,  0.1990,  0.3789, -0.1175,  0.0147]]],
       device='cuda:0', grad_fn=<CopySlices>)

In [ ]:
sigmas = torch.exp(log_sigmas)
sigmas

tensor([[[0.6436, 1.5052, 1.1337, 1.0746, 1.0932, 0.7269]],

        [[1.3248, 0.8817, 1.2623, 1.4092, 0.7228, 0.9626]],

        [[1.0620, 1.2011, 1.3048, 1.0775, 0.9677, 1.4796]],

        ...,

        [[0.8481, 0.9042, 1.1077, 0.5929, 0.9832, 0.9430]],

        [[1.1447, 1.4170, 1.5291, 1.0578, 0.7618, 1.0784]],

        [[1.2314, 0.6675, 1.2202, 1.4607, 0.8892, 1.0148]]], device='cuda:0',
       grad_fn=<ExpBackward0>)

In [ ]:
log_probs = self.agent.log_normal(actions, mus, log_sigmas)
log_probs

tensor([[[-0.6792, -1.5347, -1.2505, -2.0380, -1.1069, -0.7844]],

        [[-1.3689, -1.3149, -1.4989, -1.3144, -0.7080, -1.1587]],

        [[-0.9935, -1.9428, -1.2347, -1.4164, -0.9033, -1.9043]],

        ...,

        [[-1.4656, -1.3624, -1.2691, -0.7583, -0.9409, -1.3307]],

        [[-1.6394, -2.4802, -1.6247, -1.0102, -0.7515, -1.1696]],

        [[-2.1273, -1.9831, -1.3175, -1.3467, -0.8642, -0.9840]]],
       device='cuda:0', grad_fn=<SubBackward0>)

In [ ]:
has_positive = torch.any(log_probs > 0)
has_positive

tensor(False, device='cuda:0')

In [ ]:
log_probs = torch.clamp(log_probs, min=-100, max=0)
log_probs

tensor([[[-0.6792, -1.5347, -1.2505, -2.0380, -1.1069, -0.7844]],

        [[-1.3689, -1.3149, -1.4989, -1.3144, -0.7080, -1.1587]],

        [[-0.9935, -1.9428, -1.2347, -1.4164, -0.9033, -1.9043]],

        ...,

        [[-1.4656, -1.3624, -1.2691, -0.7583, -0.9409, -1.3307]],

        [[-1.6394, -2.4802, -1.6247, -1.0102, -0.7515, -1.1696]],

        [[-2.1273, -1.9831, -1.3175, -1.3467, -0.8642, -0.9840]]],
       device='cuda:0', grad_fn=<ClampBackward1>)

In [ ]:
rewards = torch.cat([td['reward'] for td in self._tds])
rewards

tensor([0.3109, 0.2948, 0.2953, 0.2931, 0.2526, 0.2012, 0.1472, 0.1042, 0.1294,
        0.1004, 0.0873, 0.0809, 0.0785, 0.0811, 0.0768, 0.0641, 0.0366, 0.0474,
        0.0678, 0.0882, 0.1358, 0.0781, 0.0558, 0.0445, 0.0329, 0.0332, 0.0308,
        0.0396, 0.0950, 0.1100, 0.0856, 0.0843, 0.0662, 0.0551, 0.0736, 0.0646,
        0.0237, 0.0250, 0.0429, 0.0569, 0.0595, 0.0247, 0.0185, 0.0211, 0.0261,
        0.0437, 0.0568, 0.0716, 0.1023, 0.0954, 0.0329, 0.0264, 0.0334, 0.0475,
        0.0468, 0.0409, 0.0432, 0.0450, 0.0427, 0.0457, 0.0491, 0.0514, 0.0494,
        0.0471, 0.0426, 0.0594, 0.0611, 0.0436, 0.0429, 0.0415, 0.0463, 0.0685,
        0.0433, 0.0426, 0.0430, 0.0596, 0.0448, 0.0461, 0.0442, 0.1388, 0.0749,
        0.0652, 0.0365, 0.0356, 0.0432, 0.0565, 0.0671, 0.0639, 0.0532, 0.0430,
        0.0423, 0.0369, 0.0425, 0.1093, 0.1081, 0.1127, 0.0792, 0.0506, 0.0360,
        0.0179, 0.0261, 0.0539, 0.1076, 0.1632, 0.1795, 0.2036, 0.2302, 0.2561,
        0.2736, 0.2485, 0.1814, 0.1172, 

In [ ]:
rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-6)
rewards

tensor([ 1.4229e+00,  1.3053e+00,  1.3094e+00,  1.2934e+00,  9.9750e-01,
         6.2157e-01,  2.2717e-01, -8.6642e-02,  9.7655e-02, -1.1477e-01,
        -2.1000e-01, -2.5731e-01, -2.7437e-01, -2.5551e-01, -2.8669e-01,
        -3.7982e-01, -5.8026e-01, -5.0136e-01, -3.5253e-01, -2.0388e-01,
         1.4424e-01, -2.7728e-01, -4.4015e-01, -5.2276e-01, -6.0779e-01,
        -6.0570e-01, -6.2304e-01, -5.5888e-01, -1.5371e-01, -4.4042e-02,
        -2.2233e-01, -2.3188e-01, -3.6430e-01, -4.4546e-01, -3.1044e-01,
        -3.7590e-01, -6.7449e-01, -6.6510e-01, -5.3423e-01, -4.3246e-01,
        -4.1313e-01, -6.6748e-01, -7.1278e-01, -6.9380e-01, -6.5749e-01,
        -5.2848e-01, -4.3316e-01, -3.2518e-01, -1.0058e-01, -1.5088e-01,
        -6.0796e-01, -6.5523e-01, -6.0364e-01, -5.0116e-01, -5.0612e-01,
        -5.4937e-01, -5.3251e-01, -5.1928e-01, -5.3569e-01, -5.1437e-01,
        -4.8907e-01, -4.7263e-01, -4.8695e-01, -5.0409e-01, -5.3657e-01,
        -4.1436e-01, -4.0155e-01, -5.2923e-01, -5.3

In [ ]:
rewards.mean()

tensor(5.4121e-08)

In [ ]:
import math
pi = torch.tensor([math.pi], device=self.device)
sigma = torch.exp(log_sigma)


In [ ]:
reward

tensor(0.1516)

In [ ]:
from tensordict.tensordict import TensorDict
td = TensorDict(dict(
            reward=reward.unsqueeze(0),
        ), batch_size=(1,))
td

TensorDict(
    fields={
        reward: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([1]),
    device=None,
    is_shared=False)

In [ ]:
rewards = torch.cat([td['reward']])
rewards

tensor([0.1516])

In [ ]:
actions = [td['action'] for td in self._tds_l]
actions

[tensor([[[-0.5719,  0.0060, -0.2323,  0.0000,  0.0000,  0.0000]]]),
 tensor([[[ 2.1205, -0.2813, -1.7621,  1.7935, -1.3473, -0.4830]]]),
 tensor([[[-0.7438,  0.7924, -1.1439,  0.8779, -1.0018, -0.1747]]]),
 tensor([[[-0.8687,  1.4917,  1.1869, -0.2067, -0.2375,  0.0618]]]),
 tensor([[[ 0.7312, -0.2055,  1.6420,  1.5961, -0.1922, -0.7486]]]),
 tensor([[[ 0.8980,  0.9367,  1.8458, -0.3550,  0.5398, -0.1385]]]),
 tensor([[[-0.3975, -1.7958, -0.7931,  1.4522, -0.8751,  0.0473]]]),
 tensor([[[-1.2788,  0.0930, -0.9258,  0.2446,  0.2251,  1.8892]]]),
 tensor([[[-0.2858, -0.8184, -0.7136,  0.1462,  0.3681, -0.3916]]]),
 tensor([[[-0.1187,  0.3687, -0.0695,  0.5166,  0.0033,  1.4743]]]),
 tensor([[[ 2.0865, -0.4199,  0.6356, -1.0579, -1.2160, -0.2642]]]),
 tensor([[[-1.2021,  0.8834,  0.7691,  1.9319,  0.0254, -0.5192]]]),
 tensor([[[ 0.7126,  0.1070,  0.3735,  1.3746, -0.1113,  1.2730]]]),
 tensor([[[-0.3505, -1.3240, -0.5134,  1.1177, -0.6801, -0.8094]]]),
 tensor([[[ 0.6404,  0.7395,  0.24

In [ ]:
actions_cat = torch.cat(actions)
actions_cat.shape

torch.Size([250, 1, 6])

In [ ]:
self.agent.action_filter_r

[False, False, False, True, True, True]

In [ ]:
actions_r = actions_cat[:, :, self.agent.action_filter_r]
actions_r

tensor([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

        [[ 1.7935e+00, -1.3473e+00, -4.8302e-01]],

        [[ 8.7792e-01, -1.0018e+00, -1.7474e-01]],

        [[-2.0670e-01, -2.3745e-01,  6.1809e-02]],

        [[ 1.5961e+00, -1.9220e-01, -7.4860e-01]],

        [[-3.5503e-01,  5.3979e-01, -1.3853e-01]],

        [[ 1.4522e+00, -8.7508e-01,  4.7306e-02]],

        [[ 2.4457e-01,  2.2506e-01,  1.8892e+00]],

        [[ 1.4621e-01,  3.6810e-01, -3.9157e-01]],

        [[ 5.1656e-01,  3.2753e-03,  1.4743e+00]],

        [[-1.0579e+00, -1.2160e+00, -2.6425e-01]],

        [[ 1.9319e+00,  2.5398e-02, -5.1922e-01]],

        [[ 1.3746e+00, -1.1128e-01,  1.2730e+00]],

        [[ 1.1177e+00, -6.8008e-01, -8.0940e-01]],

        [[ 3.8838e-01, -6.2765e-01,  1.7495e+00]],

        [[-6.3549e-01, -3.5441e-01,  4.7954e-01]],

        [[ 6.0069e-01, -5.3669e-01, -8.1415e-01]],

        [[-1.5638e-01, -3.5450e-01,  9.7201e-01]],

        [[-2.6603e-01, -7.5808e-01,  1.6266e+00]],

        [[ 5

In [ ]:
obs, reward, done, info = self.env.step(action)
reward

TypeError: unsupported operand type(s) for *=: 'float' and 'NoneType'

In [ ]:
self._tds_l

[TensorDict(
     fields={
         action: Tensor(shape=torch.Size([1, 1, 6]), device=cpu, dtype=torch.float32, is_shared=False),
         mu: Tensor(shape=torch.Size([1, 1, 6]), device=cpu, dtype=torch.float32, is_shared=False),
         obs: Tensor(shape=torch.Size([1, 24]), device=cpu, dtype=torch.float32, is_shared=False),
         reward: Tensor(shape=torch.Size([1, 1]), device=cpu, dtype=torch.float32, is_shared=False),
         sigma_sq: Tensor(shape=torch.Size([1, 1, 6]), device=cpu, dtype=torch.float32, is_shared=False)},
     batch_size=torch.Size([1]),
     device=None,
     is_shared=False),
 TensorDict(
     fields={
         action: Tensor(shape=torch.Size([1, 1, 1, 6]), device=cpu, dtype=torch.float32, is_shared=False),
         mu: Tensor(shape=torch.Size([1, 1, 1, 3]), device=cuda:0, dtype=torch.float32, is_shared=True),
         obs: Tensor(shape=torch.Size([1, 24]), device=cpu, dtype=torch.float32, is_shared=False),
         reward: Tensor(shape=torch.Size([1, 1]), 

In [ ]:
train_metrics, done, eval_next = {}, True, True

In [ ]:
_step = 0
_ep_idx = 0

In [ ]:
self = trainer

In [ ]:
obs = self.env.reset()

In [ ]:
td = self.to_td(obs)
td

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([1, 6]), device=cpu, dtype=torch.float32, is_shared=False),
        mu: Tensor(shape=torch.Size([1, 6]), device=cpu, dtype=torch.float32, is_shared=False),
        obs: Tensor(shape=torch.Size([1, 24]), device=cpu, dtype=torch.float32, is_shared=False),
        reward: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False),
        sigma_sq: Tensor(shape=torch.Size([1, 6]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([1]),
    device=None,
    is_shared=False)

In [ ]:
action, is_act_left = self.agent.rand_act(self.env)
action

AttributeError: 'DialecticImitation' object has no attribute 'rand_act'

In [ ]:
self.agent.device

device(type='cuda')

In [ ]:
self._tds_l = []
for _ in range(200):
    obs, reward, done, info = self.env.step(action)
    td = self.to_td(obs, action, reward)
    action, is_act_left = self.agent.rand_act(self.env)
    self._tds_l.append(td)
    # print(action, is_act_left, reward, done, info)

In [ ]:
len(self._tds_l)

200

In [ ]:
self._ep_idx = self.buffer_l.add(torch.cat(self._tds_l))
self._ep_idx

Buffer capacity: 1,000,000
Storage required: 0.13 GB
Using CUDA memory for storage.


1

In [ ]:
aaa=torch.zeros(3, device='cuda').unsqueeze(0)
aaa

tensor([[0., 0., 0.]], device='cuda:0')

In [ ]:
for _ in range(10):
    self._tds_l = []
    obs = self.env.reset()
    for _ in range(1000):
        obs, reward, done, info = self.env.step(action)
        td = self.to_td(obs, action, reward)
        action, is_act_left = self.agent.rand_act(self.env)
        self._tds_l.append(td)
        if done:
            break
    self._ep_idx = self.buffer_l.add(torch.cat(self._tds_l))
    print(self._ep_idx)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
len(self.buffer_l._buffer)

5200

In [ ]:
obs, action, reward, task = self.buffer_l.sample()

In [ ]:
obs.shape

torch.Size([4, 256, 24])

In [ ]:
obs[-1]

tensor([[-1.3881e-02, -9.9990e-01, -9.9854e-01,  ..., -1.5232e+00,
          1.4313e+01, -2.7921e-01],
        [-2.3947e-01, -9.7090e-01, -5.3082e-01,  ...,  7.9262e-03,
         -1.6760e-01,  2.3101e+00],
        [ 7.2050e-02, -9.9740e-01, -4.6628e-01,  ..., -1.1762e-01,
          3.5199e-01, -4.7712e-01],
        ...,
        [-8.7648e-02, -9.9615e-01, -8.7258e-01,  ...,  2.8130e+00,
          1.0233e+01, -4.9673e-01],
        [-4.8520e-01,  8.7441e-01,  8.4491e-01,  ..., -8.9035e-01,
          6.2931e+00, -1.8440e-02],
        [-4.7452e-01, -8.8025e-01, -5.5243e-01,  ...,  1.4005e+00,
         -4.5747e+00, -2.5643e+00]], device='cuda:0')

In [ ]:
obs[-2]

tensor([[-1.3636e-01, -9.9066e-01, -9.9569e-01,  ..., -2.5923e+00,
          7.8383e+00, -1.4251e+00],
        [-2.2628e-01, -9.7406e-01, -6.1158e-01,  ..., -8.6919e-03,
          1.1609e+00,  2.1828e-01],
        [ 1.0303e-02, -9.9995e-01, -1.8610e-01,  ...,  7.9104e+00,
         -7.5391e+00,  4.2898e+00],
        ...,
        [-1.7058e-01, -9.8534e-01, -8.6700e-01,  ..., -5.3564e+00,
          6.5812e+00,  2.1214e+01],
        [-3.9029e-01,  9.2069e-01,  9.5876e-01,  ...,  1.8240e-01,
          8.3814e+00, -3.5792e-02],
        [-4.8140e-01, -8.7650e-01, -4.5400e-01,  ..., -6.5958e+00,
          8.5040e-01,  3.8097e+00]], device='cuda:0')

In [ ]:
obs_p = obs[:-1]

In [ ]:
obs_p[-1]

tensor([[-1.3636e-01, -9.9066e-01, -9.9569e-01,  ..., -2.5923e+00,
          7.8383e+00, -1.4251e+00],
        [-2.2628e-01, -9.7406e-01, -6.1158e-01,  ..., -8.6919e-03,
          1.1609e+00,  2.1828e-01],
        [ 1.0303e-02, -9.9995e-01, -1.8610e-01,  ...,  7.9104e+00,
         -7.5391e+00,  4.2898e+00],
        ...,
        [-1.7058e-01, -9.8534e-01, -8.6700e-01,  ..., -5.3564e+00,
          6.5812e+00,  2.1214e+01],
        [-3.9029e-01,  9.2069e-01,  9.5876e-01,  ...,  1.8240e-01,
          8.3814e+00, -3.5792e-02],
        [-4.8140e-01, -8.7650e-01, -4.5400e-01,  ..., -6.5958e+00,
          8.5040e-01,  3.8097e+00]], device='cuda:0')

In [ ]:
inputs = torch.cat([obs_p, action], dim=2)
inputs.shape

torch.Size([3, 256, 30])

In [ ]:
action

tensor([[[-0.2332,  0.1174, -0.0375,  0.9871,  0.3082,  0.9214],
         [-0.6527, -0.8880, -0.9616,  0.0840,  0.9705, -0.3471],
         [-0.4501, -0.1513, -0.7177, -0.1146, -0.4243,  0.1358],
         ...,
         [-0.1581,  0.1837,  0.7980, -0.0765,  0.1488,  0.0524],
         [ 0.1693,  0.1123,  0.3253,  0.9336,  0.1717,  0.6189],
         [ 0.1103, -0.3086, -0.6671, -0.3393, -0.0569, -0.0529]],

        [[-0.2332,  0.1174, -0.0375,  0.6093,  0.7597,  0.9606],
         [-0.6527, -0.8880, -0.9616, -0.6103,  0.5416, -0.1554],
         [ 0.3978, -0.1433,  0.8257, -0.1146, -0.4243,  0.1358],
         ...,
         [-0.1581,  0.1837,  0.7980,  0.7184,  0.9668,  0.7913],
         [ 0.8526,  0.1108,  0.0430,  0.9336,  0.1717,  0.6189],
         [ 0.1103, -0.3086, -0.6671, -0.2271, -0.8366, -0.9981]],

        [[ 0.1835,  0.4421, -0.5632,  0.6093,  0.7597,  0.9606],
         [ 0.5880, -0.8386, -0.2047, -0.6103,  0.5416, -0.1554],
         [ 0.3978, -0.1433,  0.8257, -0.2450, -0.0574,  0.

In [ ]:
action.shape

torch.Size([3, 256, 6])

In [ ]:
reward.shape

torch.Size([3, 256, 1])

In [ ]:
action

tensor([[[-0.2332,  0.1174, -0.0375,  0.9871,  0.3082,  0.9214],
         [-0.6527, -0.8880, -0.9616,  0.0840,  0.9705, -0.3471],
         [-0.4501, -0.1513, -0.7177, -0.1146, -0.4243,  0.1358],
         ...,
         [-0.1581,  0.1837,  0.7980, -0.0765,  0.1488,  0.0524],
         [ 0.1693,  0.1123,  0.3253,  0.9336,  0.1717,  0.6189],
         [ 0.1103, -0.3086, -0.6671, -0.3393, -0.0569, -0.0529]],

        [[-0.2332,  0.1174, -0.0375,  0.6093,  0.7597,  0.9606],
         [-0.6527, -0.8880, -0.9616, -0.6103,  0.5416, -0.1554],
         [ 0.3978, -0.1433,  0.8257, -0.1146, -0.4243,  0.1358],
         ...,
         [-0.1581,  0.1837,  0.7980,  0.7184,  0.9668,  0.7913],
         [ 0.8526,  0.1108,  0.0430,  0.9336,  0.1717,  0.6189],
         [ 0.1103, -0.3086, -0.6671, -0.2271, -0.8366, -0.9981]],

        [[ 0.1835,  0.4421, -0.5632,  0.6093,  0.7597,  0.9606],
         [ 0.5880, -0.8386, -0.2047, -0.6103,  0.5416, -0.1554],
         [ 0.3978, -0.1433,  0.8257, -0.2450, -0.0574,  0.

In [ ]:
action_l = action[:, :, action_filter_l]

In [ ]:
action_l

tensor([[[-0.2332,  0.1174, -0.0375],
         [-0.6527, -0.8880, -0.9616],
         [-0.4501, -0.1513, -0.7177],
         ...,
         [-0.1581,  0.1837,  0.7980],
         [ 0.1693,  0.1123,  0.3253],
         [ 0.1103, -0.3086, -0.6671]],

        [[-0.2332,  0.1174, -0.0375],
         [-0.6527, -0.8880, -0.9616],
         [ 0.3978, -0.1433,  0.8257],
         ...,
         [-0.1581,  0.1837,  0.7980],
         [ 0.8526,  0.1108,  0.0430],
         [ 0.1103, -0.3086, -0.6671]],

        [[ 0.1835,  0.4421, -0.5632],
         [ 0.5880, -0.8386, -0.2047],
         [ 0.3978, -0.1433,  0.8257],
         ...,
         [-0.5654,  0.6885, -0.6125],
         [ 0.8526,  0.1108,  0.0430],
         [ 0.1696,  0.0416, -0.3987]]], device='cuda:0')

In [ ]:
obs_l = obs[:, :, obj_filter_l]
obs_l.shape

torch.Size([4, 256, 15])

In [ ]:
obs_p = obs_l[:-1]

In [ ]:
inputs = torch.cat([obs_p, action_l], dim=2)
inputs.shape

torch.Size([3, 256, 18])

In [ ]:
outputs = self.agent.model._brain_l(inputs)
outputs.shape

torch.Size([3, 256, 6])

In [ ]:
outputs

tensor([[[-0.1286,  0.1095, -0.1782, -0.0760, -0.4860,  0.0443],
         [-0.5133,  0.0200,  0.0732,  0.4288, -0.3460,  0.2054],
         [-0.7164, -0.2877,  0.3580,  0.2745, -0.2838,  0.1852],
         ...,
         [-0.2921,  0.2767,  0.3051,  0.0939, -0.1585,  0.2146],
         [ 0.0016,  0.0845, -0.0435,  0.1310,  0.3517,  0.1454],
         [-0.5692,  0.3176,  0.1097,  0.1097,  0.2101, -0.2043]],

        [[-0.2242, -0.0646, -0.3232, -0.0748, -0.6989, -0.0717],
         [-0.5786,  0.2624, -0.1827,  0.1652, -0.2234,  0.0908],
         [-0.5873,  0.0397,  0.4295,  0.3409, -0.2087,  0.4956],
         ...,
         [-0.2779,  0.2586,  0.2438,  0.0415, -0.0568,  0.0451],
         [-0.0196,  0.0322, -0.1166, -0.1400,  0.2120,  0.4093],
         [-0.4057,  0.3654,  0.1340,  0.0639,  0.1257, -0.0937]],

        [[-0.3035, -0.0170, -0.1941, -0.1729, -0.7581,  0.0045],
         [-0.3292, -0.0398,  0.0400, -0.0542,  0.2644, -0.1717],
         [-0.0890,  0.0503, -0.2042, -0.0679, -0.5531, -0.

In [ ]:
mus = outputs[:, :, :self.agent.action_dim_l]
mus

tensor([[[-0.1286,  0.1095, -0.1782],
         [-0.5133,  0.0200,  0.0732],
         [-0.7164, -0.2877,  0.3580],
         ...,
         [-0.2921,  0.2767,  0.3051],
         [ 0.0016,  0.0845, -0.0435],
         [-0.5692,  0.3176,  0.1097]],

        [[-0.2242, -0.0646, -0.3232],
         [-0.5786,  0.2624, -0.1827],
         [-0.5873,  0.0397,  0.4295],
         ...,
         [-0.2779,  0.2586,  0.2438],
         [-0.0196,  0.0322, -0.1166],
         [-0.4057,  0.3654,  0.1340]],

        [[-0.3035, -0.0170, -0.1941],
         [-0.3292, -0.0398,  0.0400],
         [-0.0890,  0.0503, -0.2042],
         ...,
         [-0.3951,  0.4228,  0.3195],
         [-0.0674,  0.1540,  0.1233],
         [-0.5047,  0.3185,  0.4071]]], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [ ]:
sigmas = outputs[:, :, self.agent.action_dim_l:]
sigmas

tensor([[[-0.0760, -0.4860,  0.0443],
         [ 0.4288, -0.3460,  0.2054],
         [ 0.2745, -0.2838,  0.1852],
         ...,
         [ 0.0939, -0.1585,  0.2146],
         [ 0.1310,  0.3517,  0.1454],
         [ 0.1097,  0.2101, -0.2043]],

        [[-0.0748, -0.6989, -0.0717],
         [ 0.1652, -0.2234,  0.0908],
         [ 0.3409, -0.2087,  0.4956],
         ...,
         [ 0.0415, -0.0568,  0.0451],
         [-0.1400,  0.2120,  0.4093],
         [ 0.0639,  0.1257, -0.0937]],

        [[-0.1729, -0.7581,  0.0045],
         [-0.0542,  0.2644, -0.1717],
         [-0.0679, -0.5531, -0.0492],
         ...,
         [ 0.3294, -0.2756, -0.0030],
         [ 0.1073, -0.0270,  0.3537],
         [ 0.3458, -0.0307,  0.6095]]], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [ ]:
import torch.nn.functional as F
sigma_sq = F.softplus(sigmas).sqrt()
sigma_sq

tensor([[[0.8099, 0.6924, 0.8459],
         [0.9646, 0.7315, 0.8950],
         [0.9164, 0.7492, 0.8888],
         ...,
         [0.8609, 0.7855, 0.8979],
         [0.8722, 0.9404, 0.8766],
         [0.8657, 0.8965, 0.7721]],

        [[0.8102, 0.6353, 0.8111],
         [0.8827, 0.7666, 0.8600],
         [0.9370, 0.7709, 0.9856],
         ...,
         [0.8450, 0.8156, 0.8461],
         [0.7909, 0.8971, 0.9584],
         [0.8518, 0.8706, 0.8046]],

        [[0.7813, 0.6199, 0.8339],
         [0.8163, 0.9133, 0.7817],
         [0.8123, 0.6741, 0.8178],
         ...,
         [0.9334, 0.7515, 0.8317],
         [0.8650, 0.8245, 0.9410],
         [0.9386, 0.8234, 1.0216]]], device='cuda:0', grad_fn=<SqrtBackward0>)

In [ ]:
eps = torch.randn(mus.size()).cuda()
eps

tensor([[[ 1.5360e+00, -5.6549e-01, -5.8816e-01],
         [ 8.0066e-01, -6.1773e-01, -1.2791e+00],
         [ 9.1077e-01, -1.3428e+00, -2.1967e+00],
         ...,
         [-5.8336e-01, -6.1134e-01,  1.7842e+00],
         [-5.8090e-01, -1.0242e+00, -2.6708e+00],
         [ 1.5530e+00,  8.9185e-01,  1.1737e+00]],

        [[-1.0217e+00,  2.2631e+00, -2.8911e+00],
         [-3.2090e+00,  5.2269e-01,  2.9869e-01],
         [-9.2925e-01,  2.7295e-01,  2.9731e-01],
         ...,
         [-1.3297e-02, -5.8256e-01,  4.2773e-01],
         [-1.0836e+00,  5.2975e-01, -4.8044e-01],
         [-1.2807e+00, -2.4445e-01, -1.4091e+00]],

        [[-1.7572e+00, -5.5766e-01,  3.7357e-01],
         [-1.0958e-03,  6.9298e-02, -1.0815e+00],
         [ 2.0388e-01, -1.6760e-01,  6.2720e-03],
         ...,
         [ 6.4166e-01, -5.6725e-01, -4.9511e-01],
         [-7.4307e-01,  1.2309e+00,  1.6127e-01],
         [ 3.7582e-02, -5.5712e-01,  7.0849e-01]]], device='cuda:0')

In [ ]:
action = mus + sigma_sq * eps
action

tensor([[[ 1.1153, -0.2820, -0.6757],
         [ 0.2590, -0.4318, -1.0716],
         [ 0.1182, -1.2937, -1.5945],
         ...,
         [-0.7943, -0.2035,  1.9071],
         [-0.5050, -0.8787, -2.3848],
         [ 0.7753,  1.1171,  1.0159]],

        [[-1.0520,  1.3730, -2.6683],
         [-3.4112,  0.6631,  0.0742],
         [-1.4580,  0.2501,  0.7226],
         ...,
         [-0.2891, -0.2165,  0.6058],
         [-0.8767,  0.5074, -0.5770],
         [-1.4967,  0.1526, -0.9998]],

        [[-1.6764, -0.3627,  0.1174],
         [-0.3301,  0.0234, -0.8053],
         [ 0.0766, -0.0626, -0.1991],
         ...,
         [ 0.2038, -0.0035, -0.0923],
         [-0.7101,  1.1688,  0.2750],
         [-0.4694, -0.1402,  1.1309]]], device='cuda:0',
       grad_fn=<AddBackward0>)

In [ ]:
import math
pi = torch.tensor([math.pi], device=self.agent.device)
pi

tensor([3.1416], device='cuda:0')

In [ ]:
entropy = 0.5 * (torch.log(2 * torch.pi * sigma_sq) + 1)
entropy

tensor([[[1.3135, 1.2351, 1.3353],
         [1.4009, 1.2626, 1.3635],
         [1.3753, 1.2746, 1.3600],
         ...,
         [1.3441, 1.2982, 1.3651],
         [1.3506, 1.3882, 1.3531],
         [1.3469, 1.3643, 1.2896]],

        [[1.3137, 1.1921, 1.3143],
         [1.3566, 1.2860, 1.3435],
         [1.3864, 1.2888, 1.4117],
         ...,
         [1.3348, 1.3170, 1.3354],
         [1.3017, 1.3646, 1.3977],
         [1.3388, 1.3497, 1.3102]],

        [[1.2955, 1.1799, 1.3281],
         [1.3175, 1.3736, 1.2958],
         [1.3150, 1.2217, 1.3184],
         ...,
         [1.3845, 1.2761, 1.3268],
         [1.3464, 1.3224, 1.3885],
         [1.3872, 1.3218, 1.4296]]], device='cuda:0', grad_fn=<MulBackward0>)

In [ ]:
self._tds_l

[TensorDict(
     fields={
         action: Tensor(shape=torch.Size([1, 6]), device=cpu, dtype=torch.float32, is_shared=False),
         obs: Tensor(shape=torch.Size([1, 24]), device=cpu, dtype=torch.float32, is_shared=False),
         reward: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False)},
     batch_size=torch.Size([1]),
     device=None,
     is_shared=False),
 TensorDict(
     fields={
         action: Tensor(shape=torch.Size([1, 6]), device=cpu, dtype=torch.float32, is_shared=False),
         obs: Tensor(shape=torch.Size([1, 24]), device=cpu, dtype=torch.float32, is_shared=False),
         reward: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False)},
     batch_size=torch.Size([1]),
     device=None,
     is_shared=False),
 TensorDict(
     fields={
         action: Tensor(shape=torch.Size([1, 6]), device=cpu, dtype=torch.float32, is_shared=False),
         obs: Tensor(shape=torch.Size([1, 24]), device=cpu, dtype=torch.

In [ ]:
self._tds_l[0]['obs']

tensor([[-0.8097,  0.5868, -0.8842,  0.4671,  0.2124, -0.9772, -0.6459, -0.7634,
         -0.4361,  0.8999, -0.3830, -0.9237, -0.9470, -0.3212,  1.2993, -0.0226,
         -0.0286, -0.7136, -2.3046, -1.3522,  1.0424,  0.6898, -4.6076, -0.7753]])

In [ ]:
obs = [td['obs'] for td in self._tds_l]
obs

[tensor([[-0.8097,  0.5868, -0.8842,  0.4671,  0.2124, -0.9772, -0.6459, -0.7634,
          -0.4361,  0.8999, -0.3830, -0.9237, -0.9470, -0.3212,  1.2993, -0.0226,
          -0.0286, -0.7136, -2.3046, -1.3522,  1.0424,  0.6898, -4.6076, -0.7753]]),
 tensor([[ -0.8265,   0.5629,  -0.9175,   0.3978,   0.4248,  -0.9053,  -0.3771,
           -0.9262,  -0.2096,   0.9778,  -0.3674,  -0.9301,  -0.3359,  -0.9419,
            1.2901,  -0.3037,  -0.1505,   1.5488,  -0.4901,  -4.0306,  -1.3473,
            9.7653,  -4.4943, -32.9662]]),
 tensor([[-0.9274,  0.3741, -0.9221,  0.3869,  0.6502, -0.7598, -0.0859, -0.9963,
           0.2977,  0.9547, -0.7205, -0.6935,  0.2583, -0.9661,  1.2551, -1.3244,
          -0.6897,  5.3465,  5.4765, -0.5248, -0.1780, 14.4875,  0.5176,  5.6941]]),
 tensor([[-0.9736,  0.2282, -0.9168,  0.3994,  0.5551, -0.8318, -0.2040, -0.9790,
           0.4730,  0.8811, -0.8925, -0.4511, -0.2882, -0.9576,  1.1629, -2.3653,
           0.1176,  0.2333,  0.7380,  2.8279,  0.0113, 

In [ ]:
obs, action, reward = [td['obs'], td['action'], td['reward'] for td in self._tds_l]
obs

SyntaxError: invalid syntax (2861983769.py, line 1)